### [ 손글씨 숫자 인식 모델 구현 ]
- 데이터셋 : mnist_train.csv, mnist_test.csv
- 학습종류 : 지동학습 - 다중클래스분류
- 학습방법 : 인공신경망기반 

[1] 모듈로딩 및 데이터 준비<hr>

In [4]:
# [1-1] 모듈로딩
import torch                                            # 텐서 및 수치과학 함수들 관련 모듈
import torch.nn as nn                                   # 신경망 층 관련 모듈
import torch.nn.functional as F                         # 신경망 함수들(AF, LF, MF) 모듈
import torch.optim as optim                             # 신경망 최적화 모듈

from torch.utils.data import Dataset, DataLoader        # pytorch의 데이터 로딩
from torch.utils.data import Subset                     # pytorch의 데이터셋 관련 모듈
from sklearn.model_selection import train_test_split
import pandas as pd


In [5]:
# [1-2] 데이터 준비
TRAIN_FILE = '../Data/mnist_train.csv'
TEST_FILE  = '../Data/mnist_test.csv'

## [1-3] 데이터 로딩
trainDF = pd.read_csv(TRAIN_FILE, header=None)
testDF  = pd.read_csv(TEST_FILE, header=None)


[2] 커스텀 데이터셋 준비 <hr>

In [6]:
## --------------------------------------------------------------------------------
## [2-1] 커스텀 데이터셋 클래스 정의
## --------------------------------------------------------------------------------
## 클래스이름 : ClfDataset
## 부모클래스 : Dataset
## 오버라이딩 : _ _init_ _(self)        : [필수] 피쳐, 타겟, [선택]행수, 컬럼수, 타겟 수...
##            _ _len_ _(self)          : len() 내장함수 실행 시 자동 호출, 샘플 수 반환
##            _ _getitem_ _(self, idx) : 인스턴스명[idx] 시 자동 호출,
##                                       idx에 해당하는 피쳐, 타겟을 텐서화 해서 반환
## --------------------------------------------------------------------------------
class ClfDataset(Dataset):

    ##- 피쳐와 타겟 저장 및 기타 속성 초기화 
    def __init__(self, dataDF):
        super().__init__()
        ## 피쳐, 타겟 초기화 필수
        self.x = dataDF[dataDF.columns[1:]].values
        self.y = dataDF[dataDF.columns[0]].values


    ##- 데이터 샘플 수 반환 메서드 : len() 함수에 자동호출됨
    def __len__(self):
        return self.x.shape[0] 
    
    ##- 인덱스에 해당하는 피쳐와 타겟 텐서 반환 메서드 : 인스턴스명[index]에 자동호출됨
    def __getitem__(self, index):
        xTS = torch.tensor(self.x[index], dtype=torch.float32)
        yTS = torch.tensor(self.y[index], dtype=torch.float32)
        return xTS, yTS



In [7]:
## --------------------------------------------------------------------------------
## [2-2] 커스텀 데이터셋 인스턴스 생성 및 사용
## --------------------------------------------------------------------------------
allDS   = ClfDataset(trainDF)
testDS  = ClfDataset(testDF)

print(f'allDS : {len(allDS)},  testDS : {len(testDS)}')

allDS : 60000,  testDS : 10000


In [ ]:
## [2-3] 학습용/검증용/테스트용 데이터셋 분리

## 학습용 데이터셋에서 타겟/라벨만 추출 
targetList = allDS.y 
dataIndexList =list( range(len(allDS)))

## 학습용/검증용 데이터셋 인덱스 분리
## -train_test_split() 함수 : train:test = 75:25 비율로 학습용, 테스트용 데이터셋 분리
##                            stratify : 분류용 데이터셋 경우 카테고리 비율 유지해서
##                                       데이터셋 분리
X_trainIdx, X_validIdx , y_train, y_valid = train_test_split( dataIndexList,
                                                              targetList,
                                                              train_size=0.8,
                                                              stratify=targetList,
                                                              random_state=10 )


In [17]:
## -----------------------------------------------------------------
## 학습용/검증용 데이터셋 생성 ===> Dataset ==> 2개 Subset 분리
## -----------------------------------------------------------------
trainDS = Subset(allDS, X_trainIdx)
validDS = Subset(allDS, X_validIdx)

print(f'allDS   : {type(allDS)},   {len(allDS)}개')
print(f'trainDS : {type(trainDS)}, {len(trainDS)}개')
print(f'validDS : {type(validDS)}, {len(validDS)}개')

allDS   : <class '__main__.ClfDataset'>,   60000개
trainDS : <class 'torch.utils.data.dataset.Subset'>, 48000개
validDS : <class 'torch.utils.data.dataset.Subset'>, 12000개
